In [1]:
import pandas as pd
import df_common as dfc

df_path = 'market_history/' + 'APEUSDT' + '.csv'
df_raw = pd.read_csv(df_path)

# open datasets and create dataframe
df = dfc.dataframe_create(df=df_raw,
                            drop=['symbol', 'tickDirection', 'trdMatchID', 
                                'side', 'grossValue', 'homeNotional', 
                                    'foreignNotional'
                                    ],
                            timestamp = 's'
                            )

# gpouping data to tameframe
minutly_price = dfc.grouping_by_time(df)
print(minutly_price)

                      Open   High    Low  Close   Volume
timestamp                                               
2022-08-01 00:00:00  6.716  6.716  6.688  6.697   6046.2
2022-08-01 00:01:00  6.697  6.716  6.687  6.716   1865.5
2022-08-01 00:02:00  6.724  6.724  6.704  6.714   7637.8
2022-08-01 00:03:00  6.718  6.718  6.706  6.716   1737.9
2022-08-01 00:04:00  6.718  6.728  6.705  6.708   3856.4
...                    ...    ...    ...    ...      ...
2022-08-03 23:55:00  7.157  7.165  7.150  7.158  15137.5
2022-08-03 23:56:00  7.153  7.171  7.153  7.169   1772.6
2022-08-03 23:57:00  7.171  7.172  7.152  7.153   5320.4
2022-08-03 23:58:00  7.153  7.165  7.150  7.165  21677.2
2022-08-03 23:59:00  7.161  7.161  7.142  7.158   4395.1

[4320 rows x 5 columns]


In [4]:
import time
import datetime

date_start = minutly_price.index[-1]
date_end = datetime.datetime.now()

date_start_with_delta = date_start + datetime.timedelta(hours=3, minutes=1)

start_with_delta_unix = int(datetime.datetime.timestamp(date_start_with_delta) * 1000)
start_unix = int(datetime.datetime.timestamp(date_start) * 1000)
end_unix = int(datetime.datetime.timestamp(date_end) * 1000)

print(start_with_delta_unix, ' | ', date_start_with_delta)
print(start_unix, ' | ', date_start)
print(end_unix, ' | ', date_end)

#date = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

1659571200000  |  2022-08-04 03:00:00
1659560340000  |  2022-08-03 23:59:00
1659607916846  |  2022-08-04 13:11:56.846694


In [14]:
from pybit import spot
from pprint import pprint

session_unauth = spot.HTTP(
    endpoint="https://api.bybit.com"
)

try:
    data = session_unauth.query_kline(
        symbol='BTCUSDT',
        interval='1m',
        startTime=start_with_delta_unix,
        endTime=end_unix
    )
except Exception as e:
    print(e)

pprint(data)

{'ext_code': None,
 'ext_info': None,
 'result': [[1659571200000,
             '22819.37',
             '22819.37',
             '22794.53',
             '22807.88',
             '2.127903',
             0,
             '48523.87457706',
             39,
             '0',
             '0'],
            [1659571260000,
             '22807.88',
             '22817.77',
             '22802.57',
             '22808.36',
             '1.367331',
             0,
             '31190.55824847',
             26,
             '0',
             '0'],
            [1659571320000,
             '22808.36',
             '22834.41',
             '22808.36',
             '22820.3',
             '0.980567',
             0,
             '22378.4164341',
             33,
             '0',
             '0'],
            [1659571380000,
             '22820.3',
             '22826.29',
             '22801.66',
             '22826.29',
             '12.549929',
             0,
             '286222.45649971',
 

In [43]:
from pybit import usdt_perpetual

session_unauth = usdt_perpetual.HTTP(
    endpoint="https://api.bybit.com"
)
data = (session_unauth.query_kline(
    symbol="BTCUSDT",
    interval=1,
    #limit=200,
    from_time=start_with_delta_unix/1000
))

In [48]:
import pandas as pd

ds = data['result']
df = pd.DataFrame(ds)
df = df.drop(['id', 'symbol', 
              'period', 'interval', 'turnover', 'open_time'], axis=1)

df = df[['start_at', 'open','high','low','close', 'volume']]

df.rename(columns={'start_at': 'timestamp', 'open': 'Open', 'high': 'High', 
                    'low': 'Low',
                   'close': 'Close', 'volume': 'Volume'}, inplace=True)

df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df = df.set_index('timestamp')

df.head()

,Open,High,Low,Close,Volume
timestamp,,,,,
2022-08-04 00:00:00,22813.0,22813.5,22785.5,22804.5,194.627
2022-08-04 00:01:00,22804.5,22809.5,22795.0,22801.0,64.120
2022-08-04 00:02:00,22801.0,22820.0,22801.0,22810.0,292.829
2022-08-04 00:03:00,22810.0,22819.0,22805.0,22819.0,32.637
2022-08-04 00:04:00,22819.0,22819.0,22789.0,22818.5,154.390


In [19]:
df[['Open', 'Close', 'High', 'Low', 'Volume']].astype(float)

,Open,Close,High,Low,Volume
timestamp,,,,,
2022-08-04 00:00:00,22819.37,22807.88,22819.37,22794.53,2.127903
2022-08-04 00:01:00,22807.88,22808.36,22817.77,22802.57,1.367331
2022-08-04 00:02:00,22808.36,22820.30,22834.41,22808.36,0.980567
2022-08-04 00:03:00,22820.30,22826.29,22826.29,22801.66,12.549929
2022-08-04 00:04:00,22826.29,22826.03,22826.29,22797.90,2.160931
...,...,...,...,...,...
2022-08-04 10:07:00,22927.58,22918.85,22936.62,22918.85,4.693695
2022-08-04 10:08:00,22918.85,22901.21,22921.19,22889.49,1.905434
2022-08-04 10:09:00,22901.21,22897.13,22916.45,22887.10,4.194086


In [20]:
print (df.dtypes)

Open      object
High      object
Low       object
Close     object
Volume    object
dtype: object


In [110]:
df_new = minutly_price.append(df)
print(df_new)

                       Open    High     Low   Close  Volume
timestamp                                                  
2022-08-01 00:00:00   6.716   6.716   6.688   6.697  6046.2
2022-08-01 00:01:00   6.697   6.716   6.687   6.716  1865.5
2022-08-01 00:02:00   6.724   6.724   6.704   6.714  7637.8
2022-08-01 00:03:00   6.718   6.718   6.706   6.716  1737.9
2022-08-01 00:04:00   6.718   6.728   6.705   6.708  3856.4
...                     ...     ...     ...     ...     ...
2022-08-04 09:48:00  7.4021   7.404  7.4005   7.404     121
2022-08-04 09:49:00   7.404  7.4133   7.404  7.4133       2
2022-08-04 09:50:00  7.4133  7.4133  7.4133  7.4133       0
2022-08-04 09:51:00  7.4133  7.4409  7.4133  7.4409  234.66
2022-08-04 09:52:00  7.4409  7.4409  7.4409  7.4409       0

[4913 rows x 5 columns]


C:\Users\daria\AppData\Local\Temp\ipykernel_10268\1016007294.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = minutly_price.append(df)


In [111]:
df_concat = pd.concat([minutly_price, df])
print(df_concat)

                       Open    High     Low   Close  Volume
timestamp                                                  
2022-08-01 00:00:00   6.716   6.716   6.688   6.697  6046.2
2022-08-01 00:01:00   6.697   6.716   6.687   6.716  1865.5
2022-08-01 00:02:00   6.724   6.724   6.704   6.714  7637.8
2022-08-01 00:03:00   6.718   6.718   6.706   6.716  1737.9
2022-08-01 00:04:00   6.718   6.728   6.705   6.708  3856.4
...                     ...     ...     ...     ...     ...
2022-08-04 09:48:00  7.4021   7.404  7.4005   7.404     121
2022-08-04 09:49:00   7.404  7.4133   7.404  7.4133       2
2022-08-04 09:50:00  7.4133  7.4133  7.4133  7.4133       0
2022-08-04 09:51:00  7.4133  7.4409  7.4133  7.4409  234.66
2022-08-04 09:52:00  7.4409  7.4409  7.4409  7.4409       0

[4913 rows x 5 columns]
